### Setup

In [25]:
# Make `scripts` importable and pull in embedder + endpoint
import sys, os

# ensure project root (one level up from notebooks/) is on sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

from scripts.process_data import LocalServerEmbeddings, EMBED_ENDPOINT

# instantiate with the endpoint from your script
embedder = LocalServerEmbeddings(endpoint=EMBED_ENDPOINT)

In [26]:
# Connect to existing ChromaDB store
from langchain_chroma import Chroma
from dotenv import load_dotenv

load_dotenv()
vectordb = Chroma(
    persist_directory="data/chroma",
    collection_name="marvel_films",
    embedding_function=embedder,
)

retriever = vectordb.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k": 3, "score_threshold": 0.1},
)

In [12]:
# Use the ChatOpenAI from langchain_openai
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-3.5-turbo",    # or model_name if that’s the signature in your version
    temperature=0.0,
)

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model_name="phi-4-mini-instruct",
    openai_api_base="http://127.0.0.1:1234/v1",
    openai_api_key="lm-studio",  # dummy key to satisfy format
)

No relevant docs were retrieved using the relevance score threshold 0.1


In [23]:
# Build the RetrievalQA chain and run your test question
from langchain.chains import RetrievalQA

qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
)

### Q&A testing

In [24]:
question = "In Avengers: Endgame, what sacrifice does Iron Man make?"
answer = qa(question)
print("Q:", question)
print("A:", answer)

Q: In Avengers: Endgame, what sacrifice does Iron Man make?
A: {'query': 'In Avengers: Endgame, what sacrifice does Iron Man make?', 'result': 'Ironman makes a significant personal and emotional sacrifice in "Avengers: Endgame". He gives away his Stark Industries fortune along with all of Tony\'s collected artifacts. This includes the Infinity Stones which he had amassed over time for safekeeping at home.\n\nTony also chooses to give up everything else, including any chance or hope that there might still be a future for himself in this new world order created by Thanos\' snap on Earth-838 (the Universe where Stark Industries was destroyed). His sacrifice is not only material but emotional as well; he gives his heart and soul into helping others even though it means giving up everything he\'s worked so hard to achieve.\n\nThis moment of selflessness serves the greater good, allowing Captain America time alone with Peggy Carter in order for them both to start a new family together. It\'s

A: {'query': 'In Avengers: Endgame, what sacrifice does Iron Man make?', 'result': 'Ironman makes a significant personal and emotional sacrifice in "Avengers: Endgame". He gives away his Stark Industries fortune along with all of Tony\'s collected artifacts. This includes the Infinity Stones which he had amassed over time for safekeeping at home.\n\nTony also chooses to give up everything else, including any chance or hope that there might still be a future for himself in this new world order created by Thanos\' snap on Earth-838 (the Universe where Stark Industries was destroyed). His sacrifice is not only material but emotional as well; he gives his heart and soul into helping others even though it means giving up everything he\'s worked so hard to achieve.\n\nThis moment of selflessness serves the greater good, allowing Captain America time alone with Peggy Carter in order for them both to start a new family together. It\'s also Tony\'s way of saying goodbye by fulfilling one final dream - coming face-to-face again as Iron Man and Thor after their years apart on Earth-838 when they reunite at Mount Wundagore.\n\nIn the end, it\'s not just about sacrificing his personal treasures or artifacts but giving up everything that makes him who he is. It\'s a bittersweet moment for Tony Stark because it means letting go of parts of himself even though he\'s doing what anyone would do in similar circumstances - selflessly putting others before themselves and hoping to make things right as best they can.\n\nTony\'s sacrifice at the end shows how much his character has grown throughout the course of "The Avengers" series. His willingness to put aside all that he holds dear for a greater good is indicative not just of Iron Man but also what it means being human - caring, sacrificing and hoping against hope even when faced with seemingly insurmountable odds.\n\nIn summary Tony\'s sacrifice in Endgame isn\'t only about giving up material possessions or artifacts like the Infinity Stones; it\'s more significantly emotional as well. He gives away everything that he holds dear to help others make things right after Thanos\' snap on Earth-838 which ultimately leads him sacrificing his personal treasures but also emotionally letting go of what makes us human - caring for each other and hoping against hope even when faced with seemingly insurmountable odds.\n'}


In [18]:
question = "Who is Ant-Man?"
answer = qa(question)
print("Q:", question)
print("A:", answer)

Q: Who is Ant-Man?
A: {'query': 'Who is Ant-Man?', 'result': 'Ant-Man is a fictional superhero appearing in comic books published by Marvel Comics. The character was created by Stan Lee, Larry Lieber, and Jack Kirby and first appeared in "Tales to Astonish" #27 in 1962. Ant-Man has the ability to shrink in size while increasing in strength, and he is able to communicate with insects. There have been multiple characters who have taken on the mantle of Ant-Man in the comics, with the most well-known being Scott Lang and Hank Pym.'}


### Conversational

In [29]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.messages import HumanMessage, SystemMessage
from google.cloud import firestore
from langchain_google_firestore import FirestoreChatMessageHistory

PROJECT_ID = "fml-project-5509d"
SESSION_ID = "user_session_new"
COLLECTION_NAME = "chat_history"

print("Initializing Firestore Client...")
client = firestore.Client(project=PROJECT_ID)

# Initialize Firestore Chat Message History
print("Initializing Firestore Chat Message History...")
chat_history = FirestoreChatMessageHistory(
    session_id=SESSION_ID,
    collection=COLLECTION_NAME,
    client=client,
)

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, just "
    "reformulate it if needed and otherwise return it as is."
)

# Create a prompt template for contextualizing questions
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Create a history-aware retriever
# This uses the LLM to help reformulate the question based on chat history
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

# Answer question prompt
# This system prompt helps the AI understand that it should provide concise answers
# based on the retrieved context and indicates what to do if the answer is unknown
qa_system_prompt = (
    "You are an assistant for question-answering tasks. Use "
    "the following pieces of retrieved context to answer the "
    "question. If you don't know the answer, just say that you "
    "don't know. Use three sentences maximum and keep the answer "
    "concise."
    "\n\n"
    "{context}"
)

# Create a prompt template for answering questions
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Create a chain to combine documents for question answering
# `create_stuff_documents_chain` feeds all retrieved context into the LLM
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# Create a retrieval chain that combines the history-aware retriever and the question answering chain
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


# Function to simulate a continual chat
def continual_chat():
    print("Start chatting with the AI! Type 'exit' to end the conversation.")
    while True:
        query = input("You: ")
        if query.lower() == "exit":
            break
        # Process the user's query through the retrieval chain
        result = rag_chain.invoke({"input": query, "chat_history": chat_history})
        # Display the AI's response
        print(f"AI: {result['answer']}")
        # Update the chat history
        chat_history.append(SystemMessage(content=result["answer"]))